In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from scipy.sparse import csr_matrix, coo_matrix
from sklearn.model_selection import ParameterGrid
from implicit.als import AlternatingLeastSquares
from implicit.nearest_neighbours import bm25_weight, tfidf_weight
from lightfm import LightFM
from lightfm.evaluation import precision_at_k, recall_at_k
from src.utils import prefilter_items
from src.metrics import precision_at_k

In [2]:
data = pd.read_csv('retail_train.csv')
item_features = pd.read_csv('product.csv')
user_features = pd.read_csv('hh_demographic.csv')
item_features.columns = [col.lower() for col in item_features.columns]
user_features.columns = [col.lower() for col in user_features.columns]
item_features.rename(columns={'product_id': 'item_id'}, inplace=True)
user_features.rename(columns={'household_key': 'user_id'}, inplace=True)
test_size_weeks = 3
data_train = data[data['week_no'] < data['week_no'].max() - test_size_weeks]
data_test = data[data['week_no'] >= data['week_no'].max() - test_size_weeks]
data_train.head(2)

,user_id,basket_id,day,item_id,quantity,sales_value,store_id,retail_disc,trans_time,week_no,coupon_disc,coupon_match_disc
0,2375,26984851472,1,1004906,1,1.39,364,-0.6,1631,1,0.0,0.0
1,2375,26984851472,1,1033142,1,0.82,364,0.0,1631,1,0.0,0.0


In [3]:
result = data_test.groupby('user_id', sort=False)['item_id'].unique().reset_index()
result.columns=['user_id', 'actual']
result.head(2)

,user_id,actual
0,338,"[840173, 1037348, 5592737, 7441679, 7442317, 1..."
1,2120,"[441768, 480433, 492628, 503227, 525168, 52575..."


In [4]:
n_items_before = data_train['item_id'].nunique()
data_train = prefilter_items(data_train, item_features)
n_items_after = data_train['item_id'].nunique()
print('Decreased # items from {} to {}'.format(n_items_before, n_items_after))

Decreased # items from 86865 to 74351


In [5]:
user_item_matrix = pd.pivot_table(data_train, 
                                  index='user_id', columns='item_id', 
                                  values='quantity',
                                  aggfunc='count', 
                                  fill_value=0
                                 )
user_item_matrix = user_item_matrix.astype(float)
sparse_user_item = csr_matrix(user_item_matrix).tocsr()
user_item_matrix.head(2)

item_id,25671,26093,26190,26426,26601,26636,26691,26738,26889,27021,...,17320911,17321083,17321092,17328261,17328742,17329473,17329749,17330255,17383227,17829232
user_id,,,,,,,,,,,,,,,,,,,,,
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [6]:
data_test = data_test[data_test['item_id'].isin(data_train['item_id'].unique())]
test_user_item_matrix = pd.pivot_table(data_test, 
                                    index='user_id', columns='item_id', 
                                    values='quantity',
                                    aggfunc='count', 
                                    fill_value=0,
                                    sort=False
                                 )
test_user_item_matrix = test_user_item_matrix.astype(float)

In [7]:
userids = user_item_matrix.index.values
itemids = user_item_matrix.columns.values
matrix_userids = np.arange(len(userids))
matrix_itemids = np.arange(len(itemids))
id_to_itemid = dict(zip(matrix_itemids, itemids))
id_to_userid = dict(zip(matrix_userids, userids))
itemid_to_id = dict(zip(itemids, matrix_itemids))
userid_to_id = dict(zip(userids, matrix_userids))

In [8]:
user_feat = pd.DataFrame(user_item_matrix.index)
user_feat = user_feat.merge(user_features, on='user_id', how='left')
user_feat.set_index('user_id', inplace=True)
item_feat = pd.DataFrame(user_item_matrix.columns)
item_feat = item_feat.merge(item_features, on='item_id', how='left')
item_feat.set_index('item_id', inplace=True)
user_feat.head(2)

,age_desc,marital_status_code,income_desc,homeowner_desc,hh_comp_desc,household_size_desc,kid_category_desc
user_id,,,,,,,
1,65+,A,35-49K,Homeowner,2 Adults No Kids,2,None/Unknown
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [9]:
user_feat_lightfm = pd.get_dummies(user_feat, columns=user_feat.columns.tolist())
item_feat_lightfm = pd.get_dummies(item_feat, columns=item_feat.columns.tolist())

In [10]:
users_ids_row = data_train['user_id'].apply(lambda x: userid_to_id[x]).values.astype(int)
items_ids_row = data_train['item_id'].apply(lambda x: itemid_to_id[x]).values.astype(int)

In [11]:
def lightfm_evaluate(no_components=10, loss='bpr', learning_rate=0.01,
                   item_alpha=0.05, user_alpha=0.05, max_sampled=100,
                   random_state=29):
    model = LightFM(no_components=no_components,
                    loss=loss,
                    learning_rate=learning_rate, 
                    item_alpha=item_alpha,
                    user_alpha=user_alpha, 
                    random_state=random_state,
                    k=5,
                    n=15,
                    max_sampled=max_sampled)
    model.fit((coo_matrix(user_item_matrix) > 0) * 1,
          sample_weight=coo_matrix(user_item_matrix),
          user_features=csr_matrix(user_feat_lightfm.values),
          item_features=csr_matrix(item_feat_lightfm.values),
          epochs=20, 
          num_threads=4)
    predictions = model.predict(user_ids=users_ids_row,
                            item_ids=items_ids_row,
                            user_features=csr_matrix(user_feat_lightfm.values),
                            item_features=csr_matrix(item_feat_lightfm.values),
                            num_threads=4)
    data_train['score'] = predictions
    predict_result = data_train[['user_id','item_id','score']][data_train.item_id != 999999]. \
                                drop_duplicates().sort_values(by=['user_id','score'], ascending=False).groupby('user_id', sort=False)['item_id']. \
                                unique().reset_index()
    df_result_for_metrics = result.merge(predict_result, on='user_id', how='inner')
    return df_result_for_metrics.apply(lambda row: precision_at_k(row['item_id'], row['actual'],k=5), axis=1).mean()

In [12]:
def find_best_params(params_grid): 
    best_score = 0
    best_params = {}
    for params_grid_dict in ParameterGrid(params_grid):   
        score = lightfm_evaluate(**params_grid_dict)
        if score > best_score:
            best_score = score
            best_params = params_grid_dict
    return best_score, best_params

In [17]:
params_grid = {
        'no_components': [30, 40, 50],
        'loss': ['bpr', 'warp'],
        'learning_rate': [0.01, 0.05, 0.1],
        'item_alpha': [0.2, 0.3],
        'user_alpha': [0.1, 0.2]
    }

In [18]:
%%time
best_score, best_params = find_best_params(params_grid)
best_score, best_params

CPU times: user 8h 27min 53s, sys: 27.1 s, total: 8h 28min 20s
Wall time: 2h 23min 26s


(0.20146914789422135,
 {'item_alpha': 0.2,
  'learning_rate': 0.01,
  'loss': 'bpr',
  'no_components': 40,
  'user_alpha': 0.1})